In [1]:
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
driver_path = "/Users/diegoandre/drivers/chromedriver"

options = webdriver.ChromeOptions()
options.set_capability(
    'goog:loggingPrefs', {'performance': 'ALL', 'browser': 'ALL'}
)

driver = webdriver.Chrome(service=ChromeService(driver_path), options=options)

In [3]:
driver.set_page_load_timeout(10)

try:
    driver.get('https://www.sofascore.com/football/match/club-puebla-tigres-uanl/PNsQN#id:13984791,tab:statistics')
except:
    pass

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [4]:
logs_raw = driver.get_log('performance')
logs = [json.loads(lr['message']) for lr in logs_raw]

In [5]:
shotmap_request_id = None
statistics_request_id = None

for log in logs:
    if log['message']['method'] == 'Network.requestWillBeSentExtraInfo':
        path = log['message']['params'].get('headers', {}).get(':path', '')
        request_id = log['message']['params']['requestId']

        if 'shotmap' in path:
            shotmap_request_id = request_id
            print(f"Found shotmap request with ID: {shotmap_request_id}")
            print(f"Shotmap Path: {path}")

        elif 'statistics' in path and '/event/' in path:
            statistics_request_id = request_id
            print(f"Found statistics request with ID: {statistics_request_id}")
            print(f"Statistics Path: {path}")

print(f"\nShotmap Request ID: {shotmap_request_id}")
print(f"Statistics Request ID: {statistics_request_id}")

Found statistics request with ID: 25072.280
Statistics Path: /api/v1/event/13984791/statistics
Found statistics request with ID: 25072.1035
Statistics Path: /api/v1/event/13984791/statistics
Found shotmap request with ID: 25072.1307
Shotmap Path: /api/v1/event/13984791/shotmap

Shotmap Request ID: 25072.1307
Statistics Request ID: 25072.1035


In [6]:
shotmap = None
if shotmap_request_id:
    try:
        shotmap_response = driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': shotmap_request_id})
        shotmap = json.loads(shotmap_response['body'])['shotmap']
        print("Shotmap data extracted successfully")
    except Exception as e:
        print(f"Error extracting shotmap: {e}")

Shotmap data extracted successfully


In [7]:
statistics = None
if statistics_request_id:
    try:
        statistics_response = driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': statistics_request_id})
        statistics = json.loads(statistics_response['body'])
        print("Statistics data extracted successfully")
    except Exception as e:
        print(f"Error extracting statistics: {e}")

Statistics data extracted successfully


In [11]:
shotmap

[{'player': {'name': 'André-Pierre Gignac',
   'slug': 'andre-pierre-gignac',
   'shortName': 'A. P. Gignac',
   'position': 'F',
   'jerseyNumber': '10',
   'userCount': 2813,
   'id': 16424,
   'fieldTranslations': {'nameTranslation': {'ar': 'أندريه-بيير جينياك',
     'hi': 'आंद्रे-पियरे गिग्नाक',
     'bn': 'আন্দ্রে-পিয়েরে গিগনাক'},
    'shortNameTranslation': {'ar': 'أ. ب. جينياك',
     'hi': 'एपी गिग्नाक',
     'bn': 'এ. পি. গিগনাক'}}},
  'isHome': True,
  'shotType': 'goal',
  'goalType': 'penalty',
  'situation': 'penalty',
  'playerCoordinates': {'x': 11.5, 'y': 50, 'z': 0},
  'bodyPart': 'right-foot',
  'goalMouthLocation': 'low-right',
  'goalMouthCoordinates': {'x': 0, 'y': 46.7, 'z': 0.6},
  'id': 5383610,
  'time': 90,
  'addedTime': 3,
  'timeSeconds': 5529,
  'draw': {'start': {'x': 50, 'y': 11.5},
   'end': {'x': 53.3, 'y': 0},
   'goal': {'x': 53.3, 'y': 99.4}},
  'reversedPeriodTime': 1,
  'reversedPeriodTimeSeconds': 771,
  'periodTimeSeconds': 129,
  'incidentType'

In [12]:
statistics

{'statistics': [{'period': 'ALL',
   'groups': [{'groupName': 'Match overview',
     'statisticsItems': [{'name': 'Ball possession',
       'home': '48%',
       'away': '52%',
       'compareCode': 2,
       'statisticsType': 'positive',
       'valueType': 'event',
       'homeValue': 48,
       'awayValue': 52,
       'renderType': 2,
       'key': 'ballPossession'},
      {'name': 'Big chances',
       'home': '5',
       'away': '0',
       'compareCode': 1,
       'statisticsType': 'positive',
       'valueType': 'event',
       'homeValue': 5,
       'awayValue': 0,
       'renderType': 1,
       'key': 'bigChanceCreated'},
      {'name': 'Total shots',
       'home': '14',
       'away': '9',
       'compareCode': 1,
       'statisticsType': 'positive',
       'valueType': 'event',
       'homeValue': 14,
       'awayValue': 9,
       'renderType': 1,
       'key': 'totalShotsOnGoal'},
      {'name': 'Goalkeeper saves',
       'home': '1',
       'away': '2',
       'compareCod